In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [7]:
data = pd.read_csv('../input/final-sarcasm-dataset/Dataset.csv')
print(data.columns)
data = data[['Text','Sarcasm']]
data.head()

Index(['Comments', 'Emojis', 'Text', 'Sarcasm'], dtype='object')


,Text,Sarcasm
0,That's how we roll . . . . . . . . #cinemaddi...,1
1,#Sarcasm,1
2,At my bro Barbara school . I'm only one can i...,1
3,And I don’t know why people get hurt #sarcas...,1
4,Yay. I see a #cold in my near #future #jokes...,1


In [8]:
data['Text'] = data['Text'].astype(str)

data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sarcasm'] == 1].size)
print(data[ data['Sarcasm'] == 0].size)

data.head()

18520
26140


,Text,Sarcasm
0,thats how we roll cinemaddicts9 talko...,1
1,sarcasm,1
2,at my bro barbara school im only one can in ...,1
3,and i dont know why people get hurt sarcasm ...,1
4,yay i see a cold in my near future jokes fun...,1


In [9]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' .,?/-!#&')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

In [12]:
embed_dim = 128
rnn_out = 512

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(SimpleRNN(rnn_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.build((None, X.shape[1]))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 22, 128)        │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ (None, 22, 128)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 512)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 585,218 (2.23 MB)

 Trainable params: 585,218 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
Y = pd.get_dummies(data['Sarcasm']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 22) (14961, 2)
(7369, 22) (7369, 2)


In [14]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20


I0000 00:00:1714789977.886788     159 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


468/468 - 13s - 28ms/step - accuracy: 0.5948 - loss: 0.6810
Epoch 2/20


W0000 00:00:1714789984.537263     156 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


468/468 - 2s - 5ms/step - accuracy: 0.6369 - loss: 0.6328
Epoch 3/20
468/468 - 2s - 4ms/step - accuracy: 0.6533 - loss: 0.6162
Epoch 4/20
468/468 - 2s - 4ms/step - accuracy: 0.6439 - loss: 0.6172
Epoch 5/20
468/468 - 2s - 4ms/step - accuracy: 0.6522 - loss: 0.6111
Epoch 6/20
468/468 - 2s - 4ms/step - accuracy: 0.6612 - loss: 0.6052
Epoch 7/20
468/468 - 2s - 5ms/step - accuracy: 0.6623 - loss: 0.6005
Epoch 8/20
468/468 - 2s - 4ms/step - accuracy: 0.6615 - loss: 0.5975
Epoch 9/20
468/468 - 2s - 4ms/step - accuracy: 0.6613 - loss: 0.6023
Epoch 10/20
468/468 - 2s - 4ms/step - accuracy: 0.6602 - loss: 0.5984
Epoch 11/20
468/468 - 2s - 4ms/step - accuracy: 0.6677 - loss: 0.5978
Epoch 12/20
468/468 - 2s - 4ms/step - accuracy: 0.6667 - loss: 0.5953
Epoch 13/20
468/468 - 2s - 4ms/step - accuracy: 0.6714 - loss: 0.5966
Epoch 14/20
468/468 - 2s - 4ms/step - accuracy: 0.6648 - loss: 0.6025
Epoch 15/20
468/468 - 2s - 4ms/step - accuracy: 0.6689 - loss: 0.5963
Epoch 16/20
468/468 - 2s - 4ms/step - a

In [15]:
model.save("Models/RNN_only_text.h5")

In [16]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

184/184 - 2s - 9ms/step - accuracy: 0.6282 - loss: 0.6442
score: 0.64
acc: 0.63


In [17]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose=0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 10.396039603960396 %
neg_acc 99.10514541387025 %


In [18]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 466ms/step
negative
